# Bernstein-Vazirani algorithm

The Bernstein-Vazirani algorithm is closely related in principle to the Deutsch-Josza algorithm, (which is also explained in another notebook in this repository).

We are given an "oracle" and promised that its output is "0" or "1" based on whether the bitwise product of the input state with a "**hidden string**" is **even** or **odd**. Our problem is to determine the "hidden string" encoded into the oracle.

A classical computer, needs to check (at least) half the possible inputs to answer the question of what the hidden string is; a quantum computer needs **1** query of the oracle to determine the same answer deterministically.

In [1]:
from pyqrack import QrackSimulator
import os

# Tell Qrack that it can use as many stabilizer qubits as it wants,
# without worrying about converting to ket for GPU:
os.environ['QRACK_MAX_CPU_QB']='-1'

# Not all cases necessarily are, but this is a stabilizer case.
# (Note that exponential encoding overhead for the oracle would be problematic to complexity theory considerations.)
oracle_qubits = [*range(100)]
num_qubits = len(oracle_qubits) + 1
hidden_bits = 1452

You may change the `hidden_bits` above to be any value that can fit within the oracle register width. Your `hidden_bits` corresponds to the oracle below:

In [2]:
def oracle(sim):
    for i in oracle_qubits:
        if ((hidden_bits >> i) & 1) > 0:
            sim.mcx([i], num_qubits - 1)

In [3]:
# Prepare the initial register state:
sim = QrackSimulator(num_qubits, isOpenCL=False)
sim.x(num_qubits - 1)
for i in range(num_qubits):
    sim.h(i)

# Make exactly one query to the oracle:
oracle(sim)

# Finish the unitary portion of the algorithm, with the result from the oracle:
for i in oracle_qubits:
    sim.h(i)

# This measurement result is always the "hidden_bits" parameter of the oracle.
result = 0
for i in oracle_qubits:
    result |= sim.m(i) << i
print("Hidden string: ", result)

Hidden string:  1452
